# Storing skymaps

## Read raw skymap files

In [2]:
import pickle
from pathlib import Path
import math

In [3]:
home_dir = "/sdf/home/o/olynn/"
raw_skymaps_dir = Path(home_dir, "skymap-to-poly-coords", "tests", "data", "raw-skymaps")

skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"
with open(skymap_path, "rb") as f:
    lsst_skymap = pickle.load(f)
lsst_skymap

*A note on rings sky map pixelization (in html comment in this cell)*
<!---## Rings sky map pixelization
*From the RingsSkyMap docstring in lsst.skymap:*

We divide the sphere into N rings of Declination, plus the two polar
caps, which sets the size of the individual tracts.  The rings are
divided in RA into an integral number of tracts of this size; this
division is made at the Declination closest to zero so as to ensure
full overlap.

Rings are numbered in the rings from south to north. The south pole cap is
``tract=0``, then the tract at ``raStart`` in the southernmost ring is
``tract=1``. Numbering continues (in the positive RA direction) around that
ring and then continues in the same fashion with the next ring north, and
so on until all reaching the north pole cap, which is
``tract=len(skymap) - 1``.

However, ``version=0`` had a bug in the numbering of the tracts: the first
and last tracts in the first (southernmost) ring were identical, and the
first tract in the last (northernmost) ring was missing. When using
``version=0``, these tracts remain missing in order to preserve the
numbering scheme.--->

## Storage options


We would like to support both inner and outer polygons for the tracts.  

Tracts are arranged in "rings", which span horizontal regions of the sky.

🔹 Inner polys:
- represent the exact boundaries of a given tract
- do not overlap

🔹 Outer polys:
- may (will?) overlap with adjacent tracts' outer polys
- are akin to margins in HATS catalogs
- will share the same upper and lower declination boundaries as the other tracts in the ring

## Option 1: Two files, explicit inner polys + reconstructable outer polys

#### Inner polys:
- Store per-tract: `ra_min`, `ra_max`, `dec_min`, `dec_max`
- Comes directly from `getRaDecRange(tract_index)`
- Requires 4 floats per tract, so for ~19k tracts:
  4 × 8 B × 19,000 ≈ ~600 KB (before YAML overhead; ~1–1.5 MB total)
- However, could compress to a `.npz` of around ~0.5–0.8 MB

#### Outer polys:
- Store per-ring:
  - `dec_min`, `dec_max`
  - `num_tracts`
  - `ra_offset`
- Reconstructs outer boundaries per tract:
    ```python
    ra_width = 360 / num_tracts
    ra_min = (ra_offset + i * ra_width) % 360
    ra_max = (ra_offset + (i + 1) * ra_width) % 360
    ```
- This is valid because outer boundaries are uniform within a ring

#### ✅ Pros
- Easy to understand and verify
- Doesn’t require any LSST WCS machinery
- Reading is fast + LSST-free

#### ❌ Cons
- Inner polygons have to be stored tract-by-tract
- Slightly more disk usage than an ultra-compressed model

### Write inner_poly and outer_poly files

In [4]:
import yaml

def write_inner_and_outer_polys(skymap, inner_path, outer_path):
    """Generate inner and outer polygon YAMLs for a RingsSkyMap.

    Parameters
    ----------
    skymap : RingsSkyMap
        The LSST RingsSkyMap object.
    inner_path : str
        Output path for per-tract inner polygon file.
    outer_path : str
        Output path for per-ring outer polygon file.
    """
    outer_rings = {}
    inner_tracts = {}
    polar_caps = {}

    for tract_index in range(len(skymap)):
        ring_num, _ = skymap.getRingIndices(tract_index)

        ra_min, ra_max, dec_min, dec_max = skymap.getRaDecRange(tract_index)
        inner_tracts[tract_index] = {
            "ra_min_deg": ra_min.asDegrees(),
            "ra_max_deg": ra_max.asDegrees(),
            "dec_min_deg": dec_min.asDegrees(),
            "dec_max_deg": dec_max.asDegrees(),
        }

        # Handle polar caps separately
        if ring_num == -1 or ring_num == skymap.config.numRings:
            polar_caps["south" if ring_num == -1 else "north"] = {
                "tract_id": tract_index,
                "dec_min_deg": dec_min.asDegrees(),
                "dec_max_deg": dec_max.asDegrees(),
            }
            continue

        outer_rings.setdefault(ring_num, {
            "tract_ids": [],
            "ra_centers": [],
        })
        outer_rings[ring_num]["tract_ids"].append(tract_index)
        # todo: should the following be a set? or, are we still just using the first element, so should just store that?
        outer_rings[ring_num]["ra_centers"].append(skymap[tract_index].getCtrCoord().getLongitude().asDegrees())

    # Build outer polygon ring YAML
    outer_yaml = {"rings": []}
    for ring_num, data in sorted(outer_rings.items()):
        tract_ids = data["tract_ids"]
        ra_centers = data["ra_centers"]
        num_tracts = len(tract_ids)
        first_tract_id = min(tract_ids)

        # Use Dec boundaries from any tract in the ring (they're shared)
        _, _, dec_min, dec_max = skymap.getRaDecRange(tract_ids[0])

        # Estimate RA offset
        ra_width = 360.0 / num_tracts
        # todo: can we make this assumption? also, can we safely assume that the tracts are evenly spaced?
        ra_offset = (ra_centers[0] - 0.5 * ra_width) % 360

        outer_yaml["rings"].append({
            "ring_index": ring_num,
            "first_tract_id": first_tract_id,
            "num_tracts": num_tracts,
            "dec_min_deg": dec_min.asDegrees(),
            "dec_max_deg": dec_max.asDegrees(),
            "ra_offset_deg": ra_offset,
        })

    if polar_caps:
        outer_yaml["polar_caps"] = polar_caps

    # Write YAMLs
    with open(inner_path, "w") as f:
        yaml.dump({"tracts": inner_tracts}, f, sort_keys=False)
    print(f"✅ Wrote exact inner polygons to: {inner_path}")

    with open(outer_path, "w") as f:
        yaml.dump(outer_yaml, f, sort_keys=False)
    print(f"✅ Wrote outer polygon metadata to: {outer_path}")


In [5]:
skymap_out_dir = "/sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/"
inner_poly_path = Path(skymap_out_dir) / "inner_polys.yaml"
outer_poly_path = Path(skymap_out_dir) / "outer_polys.yaml"

write_inner_and_outer_polys(lsst_skymap, inner_poly_path, outer_poly_path)

✅ Wrote exact inner polygons to: /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/inner_polys.yaml
✅ Wrote outer polygon metadata to: /sdf/home/o/olynn/skymap-to-poly-coords/skymaps_out/outer_polys.yaml


### Read inner_poly

In [12]:
import yaml
from lsst.sphgeom import LonLat, UnitVector3d, ConvexPolygon

def load_inner_polygons(yaml_path):
    """Load exact inner polygons from a YAML file.

    Parameters
    ----------
    yaml_path : str
        Path to inner_polys.yaml.

    Returns
    -------
    dict
        Mapping from tract ID (int) to sphgeom.ConvexPolygon.
    """
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    poly_dict = {}

    for tract_id_str, bounds in data["tracts"].items():
        tract_id = int(tract_id_str)

        # print(f"Processing tract {tract_id} with bounds: {bounds}")

        ra_min = bounds["ra_min_deg"]
        ra_max = bounds["ra_max_deg"]
        dec_min = bounds["dec_min_deg"]
        dec_max = bounds["dec_max_deg"]

        # Define corners in counter-clockwise order
        corners = [
            (ra_min, dec_min),
            (ra_max, dec_min),
            (ra_max, dec_max),
            (ra_min, dec_max),
        ]

        unit_vecs = [
            UnitVector3d(LonLat.fromDegrees(ra % 360.0, dec))
            for ra, dec in corners
        ]

        # Check for degeneracy
        unique_vecs = {tuple(round(x, 12) for x in vec) for vec in unit_vecs}
        if len(unique_vecs) < 3:
            print(f"⚠️ Skipping degenerate tract {tract_id} (probably a polar cap)")
            continue

        # for vec in unit_vecs:
        #     print(f"Unit vector: {vec}")
        poly = ConvexPolygon(unit_vecs)
        poly_dict[tract_id] = poly

    return poly_dict


In [ ]:
inner_poly_map = load_inner_polygons(inner_poly_path)

# Check if a point is inside tract 1234
from lsst.sphgeom import LonLat, UnitVector3d

pt = UnitVector3d(LonLat.fromDegrees(134.5, -2.7))
if inner_poly_map[1234].contains(pt):
    print("That point is inside tract 1234!")

⚠️ Skipping degenerate tract 0 (probably a polar cap)
⚠️ Skipping degenerate tract 18937 (probably a polar cap)


## Option 2: Reconstruct everything using projection + geometry

What you'd need to store:  

- Global:
  - `projection: "TAN"`
  - `pixel_scale_deg`: (or arcsec/pixel)
  - `tract_width`, `tract_height`: in pixels
  - `overlap_deg`: or overlap in pixels (need this for inner bounds)

- Per-tract:
  - `tract_id`
  - `center_ra`, `center_dec`
  - `rotation` (optional, usually 0)
  - `flipX` (optional, for WCS handedness)

Then, you reconstruct inner and outer polys by:
- Creating a fake `SkyWcs` from these params
- Converting pixel corners of `[0,0]` → `[width,height]` to sky coordinates
- Optionally applying overlap trimming in pixel space

✅ Pros
- Super compact: ~2 floats + a few constants per tract
- Elegant reuse of LSST projection machinery
- One unified format

❌ Cons
- Requires LSST-style WCS math to reconstruct accurately
- Introduces fragility: assumptions about projection type and tract uniformity
- Slightly harder to read/debug